<a href="https://colab.research.google.com/github/salacamifda/Cargar-y-filtrar-datos/blob/main/Data_Cleaning_Part_2_Missing_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Cleaning: Missing Values



> "Garbage in, garbage out."  

Let's clean up the garbage!

In this notebook we will:

1. Review dropping unnecessary columns and duplicates, correcting datatypes, and fixing poorly formed categories
2. Identify columns with missing values
3. Choose and implement strategies for dealing with missing values.



In [1]:
# Import Pandas
import pandas as pd

# The Data

We will be using a slightly altered version of the Titanic dataset, which is a common practice dataset for learning data science.

The dataset contains information about passengers on The Titanic as well a column describing whether they survived the accident or not.  The ultimate goal of most predictions on this dataset is to predict who will survive based on the other information.  Our goal will be to clean the data and make it ready for analysis.

Each row contains data about one passenger.

# Data Dictionary

Sometimes your data will come with a dictionary that helps you understand what the columns mean and what data types should be in them.  This is useful for cleaning the data.

| Column   | Description                                | Value Keys                                     | Type  |
|----------|--------------------------------------------|------------------------------------------------|-------|
| survival | Survival                                   | 0 = No, 1 = Yes                                | int   |
| pclass   | Ticket class                               | 1 = 1st, 2 = 2nd, 3 = 3rd                      | int   |
| sex      | Sex                                        |                                                | string   |
| Age      | Age in years                               |                                                | int   |
| sibsp    | # of siblings / spouses aboard the Titanic |                                                | int   |
| parch    | # of parents / children aboard the Titanic |                                                | int   |
| ticket   | Ticket number                              |                                                | string   |
| fare     | Passenger fare                             |                                                | float |
| cabin    | Cabin number                               |                                                | string   |
| embarked | Port of Embarkation                        | C = Cherbourg, Q = Queenstown, S = Southampton | string   |


In [2]:
# Load the Data
df = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vQ7Xta6T-hqBFfQTUMNSVdl03z74KlyecepwOK0Y4s_zv0CG6L8NrVP8tlBsnV2tI4W_cHE5NB9cduU/pub?output=csv')

## df.head()
`df.head()` is one of our most important methods.  It lets us examine the first 5 rows of our dataframe and as many columns as our maximum display columns configuration will allow.

Look below and try to decide which columns will be useful for analysis and predictions, and which columns probably do not any information that would be relevant to who might survive the catastrophic sinking of The Titanic.

In [3]:
# Examine the first 5 rows
df.head()

,Unnamed: 0,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# Remove Duplicates

We don't want any duplicate entries in our data.  This will skew our analysis and confuse our predictive models.  Duplicate entries are quite common in data that has not been cleaned.

We can use `df.duplicated()` to show whether rows are duplicates.

In [4]:
# Check for duplicates
df.duplicated()

0      False
1      False
2      False
3      False
4      False
       ...  
901     True
902     True
903     True
904     True
905     True
Length: 906, dtype: bool

Well, we don't want to have to look row by row, so let's use `.sum()` to add up all of the `True` values.  When using `.sum()`, a `True` value will evaluate to a 1.

In [5]:
# Count the duplicates
df.duplicated().sum()

15

We have 122 duplicated rows.  We can simply remove those with df.drop_duplicates().

In [6]:
# Remove duplicates
df = df.drop_duplicates()
df.duplicated().sum()

0

# Remove Unnecessary Columns

The `Unnamed: 0` column is probably a duplicate of the index.  You'll see this sometimes as Pandas tries to assign a new index when one already exists in the data.  We can drop that.

The `PassengerID` is a numbering of passengers.  Most likely the passenger number will not affect whether they passenger survided.  They also all have separate values, but are not ordered in any way and do not describe any quantitative feature of the passenger.  We will remove that.

The `Name` column is the passenger's name.  There is probably little or no correlation between a passenger's name and whether they survived.  Like `PassengerID` they are probably unique to each row and probably won't be useful for analysis or prediction.  We will remove that.

In [7]:
# Remove unnecessary columns
df = df.drop(columns=['Unnamed: 0', 'PassengerId', 'Name'])
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,3,male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,1,female,38.0,1,0,PC 17599,71.2833,C85,C
2,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,1,female,35.0,1,0,113803,53.1000,C123,S
4,0,3,male,35.0,0,0,373450,8.0500,NaN,S


## Further study of columns with df.nunique()

`.nunique()` will tell us how many unique values are in each column.  This can give us a clue about the liklihood of finding a relationship between the column value and other data.

In [8]:
# Examine the number of unique values in columns
df.nunique()

Survived      2
Pclass        3
Sex           3
Age          88
SibSp         8
Parch         7
Ticket      681
Fare        248
Cabin       147
Embarked      3
dtype: int64

`Ticket` is a category, but there are almost as many different unique values as there are rows in the dataframe.  If the Ticket data is different for most every passenger and it's not a number describing some quality of the passenger, like 'Age', then it probably won't be very useful.  We will drop that, too.

## Notice
We are using judgement calls here.  Data scientists often must use their judgement, and it's often useful to consult a subject matter expert to double check our judgement when possible.

In [9]:
# Drop the Ticket column
df = df.drop(columns='Ticket')

# Correct Data Types

## .info() 
`df.info()` is a very useful method.  It tells us a lot about our data.  What information can you gain from looking at the output below?

In [10]:
# Check columns, rows, data types, and missing values
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    object 
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Cabin     204 non-null    object 
 8   Embarked  889 non-null    object 
dtypes: float64(2), int64(3), object(4)
memory usage: 69.6+ KB


We see that there are 906 entries (rows), 9 columns remaining, and we can see how many values there are in each column and what their datatype is.

`Age` has a few missing values, `Cabin` has a lot of missing values, and `Embarked has just 2 missing values.  We will learn how to deal with those in another lesson.

Also notice that `SibSp` is an object data type.  That column represents the number of siblings and spouses a passenger has aboard with them, so it should be an integer.  Let's examine that more closely.

## Correcting Datatypes

We can use `df[column].astype(type)` to change the datatype of a column. For instance we want to change `df['SibSp']` from an object to an integer.

In [11]:
# Change the SibSp column from an object to an integer
df['SibSp'] = df['SibSp'].astype(int)

ValueError: ignored

## Errors Changing Types

Oh!  We got an error!  If we look carefully at the bottom of the error it says: `ValueError: invalid literal for int() with base 10: 'one'`

It says it can't change the string 'one' to an integer.  That means one of the values is 'one' instead of 1!

We can use `.replace` to change those, even if there are more than one of them.

In [12]:
# Replace the bad value in SibSp and try again to change the type
df['SibSp'] = df['SibSp'].replace('one', 1)
df['SibSp'] = df['SibSp'].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Cabin     204 non-null    object 
 8   Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(3)
memory usage: 69.6+ KB


`df['Age']` should also be an integer, but it is shown as a float.  This usually doesn't matter very much, as machine learning and most visualizations will display ints and floats the same.  But, what happens if we try to change the datatype?

In [13]:
df['Age'] = df['Age'].astype(int)

IntCastingNaNError: ignored

The `df['Age']` contains NA values, or missing values.  Missing values cannot be converted to integer columns.  When you see a column that should be an integer that is a float, often times this is the reason.

We will deal with the missing values in the 'Age' column in another lesson, so we will leave it as float type data for now.

# Check for Inconsistencies in Categorical Values

Another common problem with dirty data will be categories that are not uniform.  This can include misspellings, abbreviations or typos.

We can use `.value_counts()` again to examine our `object` type columns.  If we do this with numeric columns our outputs can often be too long to read easily.  It works best with categorical variables or columns.

In [14]:
# Check the unique values and counts of the 'Sex' column
df['Sex'].value_counts()

male      574
female    313
F           4
Name: Sex, dtype: int64

We see that there are both 'female' and 'F' values.  We can assume these are meant to be the same, so we will replace 'F' with 'female' so that our categories are uniform.

## Correct Incorrect Categories

In [15]:
# Standardize the values in the Sex column
df['Sex'] = df['Sex'].replace('F', 'female')
df['Sex'].value_counts()

male      574
female    317
Name: Sex, dtype: int64

# Identify Missing Values

In [16]:
df.isna().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Cabin       687
Embarked      2
dtype: int64

'Age', 'Cabin', and 'Embarked' are all missing values.

## Dropping Rows

Only 2 rows are missing values in the 'Embarked' column.  We will just drop those 2 rows since they represent a very small percentage of our data.

In [17]:
df.dropna(subset=['Embarked'], inplace=True)
df.isna().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Cabin       687
Embarked      0
dtype: int64

## Dropping Columns

The 'Cabin' column is missing a significant amount of data.

In [18]:
num_missing = df['Cabin'].isna().sum()

total_rows = df.shape[0]

percent_missing = num_missing / total_rows
print(f'{percent_missing:.2f}% of the data in the Cabin column is missing')

0.77% of the data in the Cabin column is missing


Since it contains less than 50% non-null values, we will just drop that whole column.  We won't be able to analyze if everyone in the same cabin either survived or not, but we could always bring it back if we really needed to do that with the 23% of rows that have that information.

In [19]:
df.drop(columns='Cabin', inplace=True)
df.isna().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      0
dtype: int64

Too many rows are missing data in the 'Age' column to drop the rows, but not enough to drop the column.  Instead we will impute the missing values by filling them with the median value of the ages of all of the passengers.  This will introduce a lot of errors into our dataframe, but hopefully imputing with an average age will minimize those errors.

We might be able to do more complex imputation by looking carefully at each row for clues to how old each passenger might be, but we aren't going to do that in this example.

In [20]:
age_median = df['Age'].median()
df['Age'].fillna(age_median, inplace=True)
df.isna().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

We can now also correct the datatype for `df['Age']`

In [21]:
df['Age'] = df['Age'].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 889 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  889 non-null    int64  
 1   Pclass    889 non-null    int64  
 2   Sex       889 non-null    object 
 3   Age       889 non-null    int64  
 4   SibSp     889 non-null    int64  
 5   Parch     889 non-null    int64  
 6   Fare      889 non-null    float64
 7   Embarked  889 non-null    object 
dtypes: float64(1), int64(5), object(2)
memory usage: 62.5+ KB


# NOTE:
The above imputation strategy is ONLY appropriate for preparing data for analysis.  You would not use this imputation technique when preparing data for machine learning.  You'll learn why in the next stack.

We have eliminated the missing data from the dataframe and the dataset is now ready for analysis and machine learning.

# Summary

In this lesson you reviewed data cleaning steps and learned how identify missing data, choose a strategy for dealing with it, and how to drop columns, drop rows, or use a simple imputation strategy to fill in missing data while minimizing the error you introduce into your data.

# Activity

1. Renombren la columna Unnamed: 6 a percent_change.

In [24]:
bike_df = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vQ7Xta6T-hqBFfQTUMNSVdl03z74KlyecepwOK0Y4s_zv0CG6L8NrVP8tlBsnV2tI4W_cHE5NB9cduU/pub?output=csv')

bike_df

,Unnamed: 0,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...
901,622,623,1,3,"Nakid, Mr. Sahid",male,20.0,1,1,2653,15.7417,NaN,C
902,77,78,0,3,"Moutal, Mr. Rahamin Haim",male,NaN,0,0,374746,8.0500,NaN,S
903,715,716,0,3,"Soholt, Mr. Peter Andreas Lauritz Andersen",male,19.0,0,0,348124,7.6500,F G73,S
904,623,624,0,3,"Hansen, Mr. Henry Damsgaard",male,21.0,0,0,350029,7.8542,NaN,S


In [28]:
bike_df = bike_df.rename(columns={'Unnamed: 0': 'percent_change'})

bike_df

,percent_change,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...
901,622,623,1,3,"Nakid, Mr. Sahid",male,20.0,1,1,2653,15.7417,NaN,C
902,77,78,0,3,"Moutal, Mr. Rahamin Haim",male,NaN,0,0,374746,8.0500,NaN,S
903,715,716,0,3,"Soholt, Mr. Peter Andreas Lauritz Andersen",male,19.0,0,0,348124,7.6500,F G73,S
904,623,624,0,3,"Hansen, Mr. Henry Damsgaard",male,21.0,0,0,350029,7.8542,NaN,S


2. ¿Cuántos valores faltantes hay en el conjunto de datos? ¿En qué columnas están?

In [29]:
bike_df.isna().sum()


percent_change      0
PassengerId         0
Survived            0
Pclass              0
Name                0
Sex                 0
Age               183
SibSp               0
Parch               0
Ticket              0
Fare                0
Cabin             699
Embarked            2
dtype: int64

Los datos faltantes son 884 y están en las siguientes columnas: Age(183), Cabin (699) y Embarked(2).

3. Rellenen los valores faltantes en la columna 2021 Counts con 0.


In [30]:
bike_df.head()

,percent_change,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


4. Opcional/bonus: Hay un problema desafiante pero común en el nombre para la columna de recuentos de 2019. ¿Pueden encontrarlo? Si es así, cambien el nombre de la columna de recuentos de 2019 (31 contadores) a "counts_2019" (un nombre de columna más de Python).